# 16.2 시퀀스 모델링을 위한 RNN

### 은닉 순환과 출력 순환

In [ ]:
import tensorflow as tf
tf.random.set_seed(1)
rnn_layer = tf.keras.layers.SimpleRNN(
    units=2, use_bias=True,
    return_sequences=True # 길이가 n인 시퀀스를 입력하면 출력 시퀀스가 n개의 벡터로 출력
)

rnn_layer.build(input_shape=(None, None, 5)) #배치 차원, 시퀀스, 특성 차원

w_xh, w_oo, b_h = rnn_layer.weights

print('W_xh 크기: ', w_xh.shape)
print('W_oo 크기: ', w_oo.shape)
print('b_h 크기: ', b_h.shape)

W_xh 크기:  (5, 2)
W_oo 크기:  (2, 2)
b_h 크기:  (2,)


In [ ]:
x_seq = tf.convert_to_tensor(
    [[1.0]*5, [2.0]*5, [3.0]*5],
    dtype = tf.float32)

## SimpleRNN의 출력:
output = rnn_layer(tf.reshape(x_seq, shape=(1,3,5)))

## 수동으로 출력 계산하기:
out_man = []

for t in range(len(x_seq)):
  xt = tf.reshape(x_seq[t], (1,5))
  print('타임 스텝 {} => '.format(t))
  print('      입력          :', xt.numpy())


  ht = tf.matmul(xt, w_xh) +b_h
  print('      은닉          :', ht.numpy())

  if t>0:
    prev_o = out_man[t-1]
  else:
    prev_o = tf.zeros(shape=(ht.shape))

  ot = ht + tf.matmul(prev_o, w_oo)
  ot = tf.math.tanh(ot)
  out_man.append(ot)
  print('   출력  (수동)       :', ot.numpy())
  print('    SimpleRNN   출력  :'.format(t), output[0][t].numpy())
  print()



타임 스텝 0 => 
      입력          : [[1. 1. 1. 1. 1.]]
      은닉          : [[0.41464037 0.96012145]]
   출력  (수동)       : [[0.39240566 0.74433106]]
    SimpleRNN   출력  : [0.39240566 0.74433106]

타임 스텝 1 => 
      입력          : [[2. 2. 2. 2. 2.]]
      은닉          : [[0.82928073 1.9202429 ]]
   출력  (수동)       : [[0.80116504 0.99129474]]
    SimpleRNN   출력  : [0.80116504 0.99129474]

타임 스텝 2 => 
      입력          : [[3. 3. 3. 3. 3.]]
      은닉          : [[1.243921  2.8803642]]
   출력  (수동)       : [[0.95468265 0.99930704]]
    SimpleRNN   출력  : [0.95468265 0.99930704]



# 16.3 텐서플로로 시퀀스 모델링을 위한 RNN 구현

### 16.3.1 첫번째 프로젝트: IMDb 영화 리뷰의 감성 분석 

#### 영화 리뷰 데이터 준비 

In [19]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import pandas as pd

In [20]:
# 코랩에서 실행하는 경우 다음 코드를 실행하세요.
!mkdir ../ch08
!wget https://github.com/rickiepark/python-machine-learning-book-3rd-edition/raw/master/ch08/movie_data.csv.gz -O ../ch08/movie_data.csv.gz

mkdir: cannot create directory ‘../ch08’: File exists
--2023-01-31 08:38:23--  https://github.com/rickiepark/python-machine-learning-book-3rd-edition/raw/master/ch08/movie_data.csv.gz
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/rickiepark/python-machine-learning-book-3rd-edition/master/ch08/movie_data.csv.gz [following]
--2023-01-31 08:38:23--  https://raw.githubusercontent.com/rickiepark/python-machine-learning-book-3rd-edition/master/ch08/movie_data.csv.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26521894 (25M) [application/octet-stream]
Saving to: ‘../ch08/movie_data.csv.gz’

.

In [21]:
import os
import gzip
import shutil


with gzip.open('../ch08/movie_data.csv.gz', 'rb') as f_in, open('movie_data.csv', 'wb') as f_out:
    shutil.copyfileobj(f_in, f_out)

In [22]:
df = pd.read_csv('movie_data.csv', encoding='utf-8')

df.tail()

,review,sentiment
49995,"OK, lets start with the best. the building. al...",0
49996,The British 'heritage film' industry is out of...,0
49997,I don't even know where to begin on this one. ...,0
49998,Richard Tyler is a little boy who is scared of...,0
49999,I waited long to watch this movie. Also becaus...,1


In [23]:
# 단계 1: 데이터셋 만들기
target = df.pop('sentiment')

ds_raw = tf.data.Dataset.from_tensor_slices(
    (df.values, target.values))

## 확인:
for ex in ds_raw.take(3):
    tf.print(ex[0].numpy()[0][:50], ex[1])

b'In 1974, the teenager Martha Moxley (Maggie Grace)' 1
b'OK... so... I really like Kris Kristofferson and h' 0
b'***SPOILER*** Do not read this, if you think about' 0


In [24]:
# 훈련, 테스트, 검증 데이터셋으로 나누기

tf.random.set_seed(1)
ds_raw = ds_raw.shuffle(50000, reshuffle_each_iteration=False)
ds_raw_test = ds_raw.take(25000)
ds_raw_train_valid = ds_raw.skip(25000)
ds_raw_train = ds_raw_train_valid.take(20000)
ds_raw_valid = ds_raw_train_valid.skip(20000)

In [25]:
# Counter 객체를 만들어 고유한 단어의 빈도 수집

## 2단계 : 고유 토큰 찾기
from collections import Counter
tokenizer = tfds.deprecated.text.Tokenizer()
token_counts = Counter()
for example in ds_raw_train:
  tokens = tokenizer.tokenize(example[0].numpy()[0])
  token_counts.update(tokens)
print('어휘 사전 크기: ', len(token_counts))

어휘 사전 크기:  87007


In [26]:
## 3단계 : 고유 토큰을 정수로 인코딩
encoder = tfds.deprecated.text.TokenTextEncoder(token_counts)
example_str = 'This is an example!'
print(encoder.encode(example_str))

[232, 9, 270, 1123]


In [27]:
## 3-A단계 : 변환을 위한 함수 정의
def encode(text_tensor, label):
  text = text_tensor.numpy()[0]
  encoded_text = encoder.encode(text)
  return encoded_text, label

In [28]:
## 3-B 단계: 함수를 TF 연산으로 변환하기
def encode_map_fn(text, label):
  return tf.py_function(encode, inp=[text, label], Tout=(tf.int64, tf.int64))

ds_train = ds_raw_train.map(encode_map_fn)
ds_valid = ds_raw_valid.map(encode_map_fn)
ds_test = ds_raw_test.map(encode_map_fn)

# 샘플의 크기 확인
tf.random.set_seed(1)
for example in ds_train.shuffle(1000).take(5):
  print('시퀀스 길이: ', example[0].shape)

시퀀스 길이:  (24,)
시퀀스 길이:  (179,)
시퀀스 길이:  (262,)
시퀀스 길이:  (535,)
시퀀스 길이:  (130,)


In [30]:
# 일부 데이터 추출
ds_subset = ds_train.take(8)
for example in ds_subset:
  print('개별 샘플 크기: ', example[0].shape)

# 배치 데이터 만들기
ds_batched = ds_subset.padded_batch(
    4, padded_shapes=([-1], [])
)

for batch in ds_batched:
  print('배치 차원: ', batch[0].shape)

개별 샘플 크기:  (119,)
개별 샘플 크기:  (688,)
개별 샘플 크기:  (308,)
개별 샘플 크기:  (204,)
개별 샘플 크기:  (326,)
개별 샘플 크기:  (240,)
개별 샘플 크기:  (127,)
개별 샘플 크기:  (453,)
배치 차원:  (4, 688)
배치 차원:  (4, 453)


In [31]:
# 32의 미니 배치로 나누기
train_data = ds_train.padded_batch(
    32, padded_shapes=([-1], [])
)
valid_data = ds_valid.padded_batch(
    32, padded_shapes=([-1], [])
)
test_data = ds_test.padded_batch(
    32, padded_shapes=([-1], [])
)

#### 문장 인코딩을 위한 임베딩 층 

In [33]:
from tensorflow.keras.layers import Embedding
model = tf.keras.Sequential()
model.add(Embedding(input_dim=100, #batch size X input_length차원을 가진 랭크 2
                    output_dim=6,  #batch size  X input_length X embedding_dim(임베딩 특성의 크기) 
                    input_length=20, #시퀀스 길이
                    name='embed-layer'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embed-layer (Embedding)     (None, 20, 6)             600       
                                                                 
Total params: 600
Trainable params: 600
Non-trainable params: 0
_________________________________________________________________


#### RNN 모델 만들기

In [34]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import SimpleRNN
from tensorflow.keras.layers import Dense

model = Sequential()
model.add(Embedding(input_dim=1000, output_dim=32))
model.add(SimpleRNN(32, return_sequences=True))
model.add(SimpleRNN(32))
model.add(Dense(1))
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 32)          32000     
                                                                 
 simple_rnn (SimpleRNN)      (None, None, 32)          2080      
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, 32)                2080      
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 36,193
Trainable params: 36,193
Non-trainable params: 0
_________________________________________________________________


#### 감성 분석 작업을 위한 RNN 모델 만들기 

In [37]:
embedding_dim = 20
vocab_size = len(token_counts) + 2
tf.random.set_seed(1)

## 모델 생성
bi_lstm_model = tf.keras.Sequential([
    tf.keras.layers.Embedding(
        input_dim=vocab_size,
        output_dim=embedding_dim,
        name='embed-layer'),
    
    tf.keras.layers.Bidirectional(
        tf.keras.layers.LSTM(64, name='lstm-layer'),
        name='bidir-lstm'), 

    tf.keras.layers.Dense(64, activation='relu'),
    
    tf.keras.layers.Dense(1, activation='sigmoid')
])

bi_lstm_model.summary()

## 컴파일과 훈련:
bi_lstm_model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-3),
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
    metrics=['accuracy'])

history = bi_lstm_model.fit(
    train_data, 
    validation_data=valid_data, 
    epochs=10)

## 테스트 데이터에서 평가
test_results= bi_lstm_model.evaluate(test_data)
print('테스트 정확도: {:.2f}%'.format(test_results[1]*100))

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embed-layer (Embedding)     (None, None, 20)          1740180   
                                                                 
 bidir-lstm (Bidirectional)  (None, 128)               43520     
                                                                 
 dense_3 (Dense)             (None, 64)                8256      
                                                                 
 dense_4 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1,792,021
Trainable params: 1,792,021
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
625/625 [==============================] - 61s 86ms/step - loss: 0.4780 - accuracy: 0.7613 - val_loss: 0.3455 - val_accuracy: 0.8622
Epoch 2/10
625/625 [============

In [38]:
## SimpleRNN에서 납득할 수 있는 수준의 예측 성능을 얻기 위해 시퀀스 길이 줄이기

# 헬퍼 함수preprocess_datasets()를 만들어 전처리 과정 연결

from collections import Counter
def preprocess_datasets(
    ds_raw_trian,
    ds_raw_valid,
    ds_raw_test,
    max_seq_length=None,
    batch_size=32):
  
  ## 1단계 : 데이터셋 만들기 이미 완료
  ## 2단계 : 고유 토큰 찾기
  tokenizer = tfds.deprecated.text.Tokenizer()
  token_counts = Counter()

  for example in ds_raw_train:
    tokens = tokenizer.tokenize(example[0].numpy()[0])
    if max_seq_length is not None:
      tokens = tokens[-max_seq_length:]
    token_counts.update(tokens)

  print('어휘 사전 크기: ', len(token_counts))
  

  ## 3단계 : 텍스트 인코딩하기
  encoder = tfds.deprecated.text.TokenTextEncoder(token_counts)

  def encode(text_tensor, label):
    text = text_tensor.numpy()[0]
    encoded_text = encoder.encode(text)
    if max_seq_length is not None:
      encoded_text = encoded_text[-max_seq_length:]
    return encoded_text, label

  def encode_map_fn(text, label):
    return tf.py_function(encode, inp=[text, label], Tout=(tf.int64, tf.int64))

  ds_train = ds_raw_train.map(encode_map_fn)
  ds_vaid = ds_raw_valid.map(encode_map_fn)
  ds_test = ds_raw_test.map(encode_map_fn)

  ## 4단계 : 배치 데이터 만들기
  train_data = ds_train.padded_batch(batch_size, padded_shapes=([-1], []))
  valid_data = ds_valid.padded_batch(batch_size, padded_shapes=([-1],[]))
  test_data = ds_test.padded_batch(batch_size, padded_shapes=([-1], []))

  return (train_data, valid_data, test_data, len(token_counts))





In [39]:
# build_run_model() 헬퍼 함수 정의

from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import SimpleRNN
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import GRU

def build_rnn_model(embedding_dim, vocab_size,
                    recurrent_type='SimpleRNN',
                    n_recurrent_units=64,
                    n_recurrent_layers=1,
                    bidirectional=True):

    tf.random.set_seed(1)

    # 모델 생성
    model = tf.keras.Sequential()
    
    model.add(
        Embedding(
            input_dim=vocab_size,
            output_dim=embedding_dim,
            name='embed-layer')
    )
    
    for i in range(n_recurrent_layers):
        return_sequences = (i < n_recurrent_layers-1)
            
        if recurrent_type == 'SimpleRNN':
            recurrent_layer = SimpleRNN(
                units=n_recurrent_units, 
                return_sequences=return_sequences,
                name='simprnn-layer-{}'.format(i))
        elif recurrent_type == 'LSTM':
            recurrent_layer = LSTM(
                units=n_recurrent_units, 
                return_sequences=return_sequences,
                name='lstm-layer-{}'.format(i))
        elif recurrent_type == 'GRU':
            recurrent_layer = GRU(
                units=n_recurrent_units, 
                return_sequences=return_sequences,
                name='gru-layer-{}'.format(i))
        
        if bidirectional:
            recurrent_layer = Bidirectional(
                recurrent_layer, name='bidir-'+recurrent_layer.name)
            
        model.add(recurrent_layer)

    model.add(tf.keras.layers.Dense(64, activation='relu'))
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
    
    return model

In [40]:
## SimpleRNN 층 하나를 가진 모델을 최대 길이가 토큰 100개인 시퀀스로 훈련

batch_size = 32
embedding_dim = 20
max_seq_length = 100

train_data, valid_data, test_data, n = preprocess_datasets(
    ds_raw_train, ds_raw_valid, ds_raw_test,
    max_seq_length=max_seq_length,
    batch_size = batch_size
)

vocab_size = n+2
rnn_model = build_rnn_model(
    embedding_dim, vocab_size,
    recurrent_type = 'SimpleRNN',
    n_recurrent_units=64,
    n_recurrent_layers=1,
    bidirectional=True
)
rnn_model.summary()

어휘 사전 크기:  58063
Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embed-layer (Embedding)     (None, None, 20)          1161300   
                                                                 
 bidir-simprnn-layer-0 (Bidi  (None, 128)              10880     
 rectional)                                                      
                                                                 
 dense_5 (Dense)             (None, 64)                8256      
                                                                 
 dense_6 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1,180,501
Trainable params: 1,180,501
Non-trainable params: 0
_________________________________________________________________


In [45]:
rnn_model.compile(optimizer=tf.keras.optimizers.Adam(1e-3),
                  loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
                  metrics=['accuracy'])

history = rnn_model.fit(
    train_data,
    validation_data = valid_data,
    epochs=10
)

results = rnn_model.evaluate(test_data)
print('테스트 정확도: {:.2f}%'.format(results[1]*100))

Epoch 1/10
625/625 [==============================] - 142s 225ms/step - loss: 0.1272 - accuracy: 0.9477 - val_loss: 1.0376 - val_accuracy: 0.5066
Epoch 2/10
625/625 [==============================] - 143s 229ms/step - loss: 0.0650 - accuracy: 0.9769 - val_loss: 2.1647 - val_accuracy: 0.5088
Epoch 3/10
625/625 [==============================] - 142s 228ms/step - loss: 0.0228 - accuracy: 0.9911 - val_loss: 2.3633 - val_accuracy: 0.5070
Epoch 4/10
625/625 [==============================] - 175s 280ms/step - loss: 0.0195 - accuracy: 0.9927 - val_loss: 2.9121 - val_accuracy: 0.5014
Epoch 5/10
625/625 [==============================] - 178s 285ms/step - loss: 0.0250 - accuracy: 0.9912 - val_loss: 1.6936 - val_accuracy: 0.4984
Epoch 6/10
625/625 [==============================] - 153s 245ms/step - loss: 0.0296 - accuracy: 0.9890 - val_loss: 1.6208 - val_accuracy: 0.5080
Epoch 7/10
625/625 [==============================] - 178s 285ms/step - loss: 0.0066 - accuracy: 0.9979 - val_loss: 1.8861 -

## 두 번째 프로젝트: 텐서플로로 글자 단위 언어 모델 구현

#### 데이터셋 전처리

In [46]:
# 코랩에서 실행할 경우 다음 코드를 실행해 주세요.
!wget https://raw.githubusercontent.com/rickiepark/python-machine-learning-book-3rd-edition/master/ch16/1268-0.txt

--2023-01-31 10:30:29--  https://raw.githubusercontent.com/rickiepark/python-machine-learning-book-3rd-edition/master/ch16/1268-0.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1171600 (1.1M) [text/plain]
Saving to: ‘1268-0.txt’

1268-0.txt          100%[===================>]   1.12M  --.-KB/s    in 0.006s  

2023-01-31 10:30:29 (182 MB/s) - ‘1268-0.txt’ saved [1171600/1171600]



In [47]:
import numpy as np

## 텍스트 읽고 전처리하기
with open('1268-0.txt', 'r', encoding='UTF8') as fp:
    text=fp.read()
    
start_indx = text.find('THE MYSTERIOUS ISLAND')
end_indx = text.find('End of the Project Gutenberg')
print(start_indx, end_indx)

text = text[start_indx:end_indx]
char_set = set(text)
print('전체 길이:', len(text))
print('고유한 문자:', len(char_set))

567 1112917
전체 길이: 1112350
고유한 문자: 80


In [48]:
## 문자를 정수로 매핑하기, 역매핑하기

chars_sorted =sorted(char_set)
char2int = {ch:i for i, ch in enumerate(chars_sorted)}
char_array = np.array(chars_sorted)

text_encoded = np.array(
    [char2int[ch] for ch in text],
     dtype=np.int32)
print('인코딩된 텍스트 크기: ', text_encoded.shape)

print(text[:15], '       ==인코딩 ==> ', text_encoded[:15])
print(text_encoded[15:21], ' == 디코딩 ==> ', 
      ''.join(char_array[text_encoded[15:21]]))


인코딩된 텍스트 크기:  (1112350,)
THE MYSTERIOUS         ==인코딩 ==>  [44 32 29  1 37 48 43 44 29 42 33 39 45 43  1]
[33 43 36 25 38 28]  == 디코딩 ==>  ISLAND


In [50]:
# 배열을 사용하여 텐서플로 데이터셋 만들기

import tensorflow as tf

ds_text_encoded = tf.data.Dataset.from_tensor_slices(text_encoded)

for ex in ds_text_encoded.take(5):
  print('{} -> {}'.format(ex.numpy(), char_array[ex.numpy()]))



44 -> T
32 -> H
29 -> E
1 ->  
37 -> M


In [51]:
# 텍스트 생성과 변환

seq_length = 40
chunk_size = seq_length + 1
ds_chunks = ds_text_encoded.batch(chunk_size, drop_remainder=True)

## x & y를 나누기 위한 함수 정의
def split_input_target(chunk):
  input_seq = chunk[:-1]
  target_seq = chunk[1:]
  return input_seq, target_seq

ds_sequences = ds_chunks.map(split_input_target)


for example in ds_sequences.take(2):
  print('입력 (x): ', repr(''.join(char_array[example[0].numpy()])))
  print('타깃 (y): ', repr(''.join(char_array[example[1].numpy()])))
  print()

입력 (x):  'THE MYSTERIOUS ISLAND ***\n\n\n\n\nProduced b'
타깃 (y):  'HE MYSTERIOUS ISLAND ***\n\n\n\n\nProduced by'

입력 (x):  ' Anthony Matonak, and Trevor Carlson\n\n\n\n'
타깃 (y):  'Anthony Matonak, and Trevor Carlson\n\n\n\n\n'



In [52]:
# 미니 배치로 나누기

BATCH_SIZE= 64
BUFFER_SIZE = 10000
ds= ds_sequences.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

#### 문자 수준의 RNN 모델 만들기

In [53]:
def build_model(vocab_size, embedding_dim, rnn_units):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim),
        tf.keras.layers.LSTM(
            rnn_units, return_sequences=True),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model


# 매개변수 설정

charset_size = len(char_array)
embedding_dim = 256
rnn_units = 512

tf.random.set_seed(1)

model = build_model(
    vocab_size = charset_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, None, 256)         20480     
                                                                 
 lstm (LSTM)                 (None, None, 512)         1574912   
                                                                 
 dense_7 (Dense)             (None, None, 80)          41040     
                                                                 
Total params: 1,636,432
Trainable params: 1,636,432
Non-trainable params: 0
_________________________________________________________________


In [54]:
# 모델 훈련

model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))

model.fit(ds,epochs=20)

Epoch 1/20
424/424 [==============================] - 8s 15ms/step - loss: 2.3203
Epoch 2/20
424/424 [==============================] - 7s 16ms/step - loss: 1.7480
Epoch 3/20
424/424 [==============================] - 7s 14ms/step - loss: 1.5487
Epoch 4/20
424/424 [==============================] - 7s 14ms/step - loss: 1.4342
Epoch 5/20
424/424 [==============================] - 7s 14ms/step - loss: 1.3608
Epoch 6/20
424/424 [==============================] - 6s 14ms/step - loss: 1.3098
Epoch 7/20
424/424 [==============================] - 7s 15ms/step - loss: 1.2713
Epoch 8/20
424/424 [==============================] - 7s 14ms/step - loss: 1.2407
Epoch 9/20
424/424 [==============================] - 6s 14ms/step - loss: 1.2142
Epoch 10/20
424/424 [==============================] - 7s 14ms/step - loss: 1.1926
Epoch 11/20
424/424 [==============================] - 7s 14ms/step - loss: 1.1729
Epoch 12/20
424/424 [==============================] - 7s 14ms/step - loss: 1.1551
Epoch 13/20
4

#### 평가 단계: 새로운 텍스트 생성

In [56]:
tf.random.set_seed(1)
logits =[[1.0, 1.0, 1.0]]
print('확률: ', tf.math.softmax(logits).numpy()[0])

samples = tf.random.categorical( # 모델이 출력한 로짓을 기반으로 문자를 생성 
    logits=logits, num_samples=10
)

tf.print(samples.numpy())

확률:  [0.33333334 0.33333334 0.33333334]
array([[1, 2, 0, 1, 0, 1, 1, 2, 1, 1]])


In [57]:
tf.random.set_seed(1)
logits = [[1.0,1.0, 3.0]]
print('확률: ', tf.math.softmax(logits).numpy()[0])
samples = tf.random.categorical(logits=logits, num_samples=10)
tf.print(samples.numpy())



확률:  [0.10650698 0.10650698 0.78698605]
array([[2, 2, 0, 2, 2, 2, 2, 2, 1, 2]])


In [58]:
# 짧은 시작 문자열 starting_str을 받아 새로운 generated_str을 생성하는 sample() 함수

# generated_str : 초기 입력값으로 설정
#max_input_length 크기의 문자열을 선택하여 정수 시퀀스 encoded_input 으로 인코딩
# encoded_input을 RNN 모델에 전달하여 로짓을 계산
# 동일한 길이의 로짓 시퀀스 출력 

# 출력 loits의 마지막 원소만 tf.random.categorical() 함수로 전달하여 새로운 샘플 생성 
# 새로운 샘플을 문자로 변환하고 생성된 무낮열 generated_text 끝에 추가하여 길이를 1만큼 늘림
# 과정 반복
# 지정된 문자 길이만큼 생성될 때까지 generated_text에서 마지막 max_input_length개의 문자를 선택하고 이를 사용하여 새로운 문자를 생성
# 새로운 원소를 만들기 위해 생성된 시퀀스를 입력으로 사용하는 과정 : 자기회귀 

def sample(model, starting_str, 
           len_generated_text=500, 
           max_input_length=40,
           scale_factor=1.0):
    encoded_input = [char2int[s] for s in starting_str]
    encoded_input = tf.reshape(encoded_input, (1, -1))

    generated_str = starting_str

    model.reset_states()
    for i in range(len_generated_text):
        logits = model(encoded_input)
        logits = tf.squeeze(logits, 0)

        scaled_logits = logits * scale_factor
        new_char_indx = tf.random.categorical(
            scaled_logits, num_samples=1)
        
        new_char_indx = tf.squeeze(new_char_indx)[-1].numpy()    

        generated_str += str(char_array[new_char_indx])
        
        new_char_indx = tf.expand_dims([new_char_indx], 0)
        encoded_input = tf.concat(
            [encoded_input, new_char_indx],
            axis=1)
        encoded_input = encoded_input[:, -max_input_length:]

    return generated_str

tf.random.set_seed(1)
print(sample(model, starting_str='The island'))

The island was eggy extremets, that if he fell I was setting so tranquil, who joured the case to repoars from these points, which which made their
former side. All that syel into this plan a harding care, and a valley--across a
large
thousand clouds, which there is never good and water a part of the edge of the electrily tremenly, understood
that he should think, sond Ayrton, and there were thrown with heavy six layfuls,
while tem in any equal intelligent and re-established by the time was scanced
to turn


In [59]:
logits =np.array([1.0, 1.0, 3.0])
print('스케일 조정 전의 확률: ', tf.math.softmax(logits).numpy()[0])
print('0.5배 조정 후 확률: ', tf.math.softmax(0.5*logits).numpy()[0])
print('0.1배 조정 후 확률: ', tf.math.softmax(0.1*logits).numpy()[0])

스케일 조정 전의 확률:  0.10650697891920076
0.5배 조정 후 확률:  0.21194155761708544
0.1배 조정 후 확률:  0.31042377345300565


In [60]:
# alpha = 2.0 -> 예측 가능성이 높아짐 

tf.random.set_seed(1)
print(sample(model, starting_str='The island', 
             scale_factor=2.0))

The island was extremely stream and the last season were in the lake, and were to be feared that the car before the work had at the same time a patience, since the engineer had been saved and a little border of the cart, which was sufficient to see in the corral in the work of the sea, and the first event with near the trees were suddenly transformed into the sea.

“Well, my friend,” said the reporter, “and the influence of the engineer and his companions were still the brig, and at the same time when the


In [61]:
# alpha = 0.5 -> 무작위성이 높아짐

tf.random.set_seed(1)
print(sample(model, starting_str='The island', 
             scale_factor=0.5))

The island was egbles likely going
away.
Pamp, Herbert! ciff swelland,
mercyrt-lip,--tho case twirty tho favoce,
supportahes abon hhis a;
unferwind whey weig here, even Herbert?

Dureap vern his irrecamb; though atlasquegrals could ice-ferpehousces del shared, ture throughoughar fust,! at
alful them, Spilett after!”

All Ayrto lithus inscredence
eel? Very newing, whnouming as fare hurn--huse my lun. He hall?” :
illy oursquitly yed in any equal further. Larters were well
by mercy. Whilds, Harding sworA9 hi
